In [1]:
wd <- dirname(getwd())
source(paste0(wd,"/mission_control/treasure_map.R"))
source(paste0(wd,"/mission_control/clinical_help.R"))

library(dplyr)
library(tidyr)

Warning message:
“package ‘dplyr’ was built under R version 3.6.3”
Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Warning message:
“package ‘tidyr’ was built under R version 3.6.3”

#### 0 - read in clinical data with dups 

In [2]:
clinical <- read.csv(paste0( TMP_DIR,'clinical_short.csv'), header=TRUE, sep = ",", stringsAsFactors=FALSE)

#### 1 - De-dup CPI patient data 

In [3]:
cpi <- (
    clinical 
        %>% filter(clinical_post_contains_Immunotherapy == "True")
        %>% group_by(patientIdentifier) 
        %>% mutate(ct = n(), sample_order = rank(sampleId)) 
        %>% ungroup()
        %>% arrange(patientIdentifier)
)  
non_dup <- cpi %>% filter( ct == 1) %>% select(-ct, -sample_order)
keep_samples <- c("CPCT02010503TII", "CPCT02020670T", "CPCT02020958TII", "CPCT02050073TII", "CPCT02050276TII", "CPCT02080266T")
dup <- cpi %>% filter(sampleId %in% keep_samples) %>% select(-ct, -sample_order)
cpis <- rbind(non_dup, dup)

#### 2 - De-dup non-CPI patients

In [4]:
non_cpis <- (
    clinical 
        %>% filter(clinical_post_contains_Immunotherapy != "True", 
                   !(patientIdentifier %in% cpi$patientIdentifier))
        %>% group_by(patientIdentifier) 
        %>% mutate(sample_order = rank(desc(sampleId)) )
        %>% ungroup()
        %>% filter(sample_order == 1)
        %>% select(-sample_order))

In [5]:
clinical <- rbind(cpis,non_cpis)

#### 3 - Clean and derive more features

In [6]:
clinical$clinical_number_pretreatment <- unlist(lapply(clinical$ID_pre_name, function(i) length(strsplit(i, "/")[[1]])))

####  Treatment mechanism indicators
- Only indicators for mechanisms with atleast 20 samples
- New addition after immune biomarkers project

In [7]:
trt_mechanisms <- unique(unlist(lapply(clinical$ID_post_mechanism, function(i) strsplit(i, "/")[[1]])))
                                       
sizes <- data.frame()
for (i in c(trt_mechanisms, "Pyrimidine", "GnRH")){
    size <- data.frame( "trt_mechanism" = i, ct = nrow(clinical %>% filter(grepl(i, ID_post_mechanism))))
    sizes <- rbind(sizes, size) 
}
sizes <- sizes %>% arrange(desc(ct)) %>% filter(trt_mechanism != "inhibitor") %>% filter(ct > 20)
                                       
for( i in sizes$trt_mechanism ){
    clean_mech <- gsub("[^[:alnum:] ]","",i)
    field_name <- paste0("clinical_trt_mechanism_contains_", clean_mech )
    clinical[,field_name] <- ifelse( grepl(i, clinical$ID_post_mechanism), 1, 0)
}                                        

#### Add subtype indicators
- Only noteworthy subtypes with sufficient sample size are in breast and lung

In [8]:
subtypes <- (
    clinical 
        %>% filter(clinical_meta_primaryTumorLocation %in% c("Breast", "Lung")) 
        %>% group_by(ID_meta_primaryTumorSubType) 
        %>% summarise(ct = n()) 
        %>% arrange(desc(ct)) 
        %>% filter(ct > 20)
        %>% pull(ID_meta_primaryTumorSubType)
)

In [9]:
for( i in subtypes ){
    clean_subtype <- gsub(" ", "_", gsub("[^[:alnum:] ]","_",i))
    print(clean_subtype)
    field_name <- paste0("clinical_subtype_", clean_subtype )
    clinical[,field_name] <- ifelse( grepl(i, clinical$ID_meta_primaryTumorSubType), 1, 0)
}        

[1] "ER_positive_HER2_negative"
[1] "Non_small_cell_carcinoma"
[1] "ER_positive_HER2_positive"
[1] "Triple_negative"
[1] "Small_cell_carcinoma"
[1] "ER_negative_HER2_positive"


#### Tumor location grouping

In [10]:
#clinical %>% filter(clinical_trt_mechanism_contains_Platinum == 1) %>% group_by( clinical_meta_primaryTumorLocation ) %>% summarise(ct = n()) %>% arrange(desc(ct))

#### More features added 
- Initially all from immunebiomarkers project 

In [11]:
clinical$clinical_tumor_location_group <- unlist(lapply(clinical$clinical_tumor_location_group, tissue))
clinical$clinical_meta_primaryTumorLocation <- unlist(lapply(clinical$clinical_meta_primaryTumorLocation, tissue))
clinical$clinical_meta_consolidatedTreatmentType <- unlist(lapply(clinical$clinical_meta_consolidatedTreatmentType, therapy))
clinical$clinical_meta_hasRadiotherapyPreTreatment <- ifelse(clinical$clinical_meta_hasRadiotherapyPreTreatment == "Yes", 1, 0)
clinical$clinical_meta_hasSystemicPreTreatment2 <- ifelse(clinical$clinical_meta_hasSystemicPreTreatment == "Yes", 1, 0)
clinical$clinical_meta_gender <- ifelse(clinical$clinical_meta_gender == "female", 1, 0)
clinical$clinical_pre_contains_Chemotherapy <- ifelse(clinical$clinical_pre_contains_Chemotherapy == "True", 1, 0)
clinical$clinical_post_contains_Chemotherapy <- ifelse(clinical$clinical_post_contains_Chemotherapy == "True", 1, 0)
clinical$clinical_pre_contains_Hormonal <- ifelse(clinical$clinical_pre_contains_Hormonal == "True", 1, 0)
clinical$clinical_post_contains_Hormonal <- ifelse(clinical$clinical_post_contains_Hormonal == "True", 1, 0)
clinical$clinical_pre_contains_Immunotherapy <- ifelse(clinical$clinical_pre_contains_Immunotherapy == "True", 1, 0)
clinical$clinical_pre_contains_Targeted <- ifelse(clinical$clinical_pre_contains_Targeted == "True", 1, 0)
clinical$clinical_post_contains_Targeted <- ifelse(clinical$clinical_post_contains_Targeted == "True", 1, 0)
clinical$clinical_pre_treated <- as.numeric(clinical$clinical_meta_hasRadiotherapyPreTreatment + clinical$clinical_meta_hasSystemicPreTreatment2 > 0)
clinical$clinical_cpi_mechanism <- get_cpi_mechanism(clinical$clinical_meta_treatment)
clinical$clinical_cpi_mechanism2 <- ifelse( clinical$clinical_cpi_mechanism %in% c("PD1", "PDL1"), "PD", clinical$clinical_cpi_mechanism)
clinical$clinical_cpi_mechanism3 <- ifelse( clinical$clinical_cpi_mechanism == "multiple", 1, 0)
clinical$clinical_systemic_composite <- (
        ifelse(clinical$clinical_meta_hasSystemicPreTreatment2 == 0, 
               365*10, 
               clinical$clinical_pre_to_post_treatment_time
              )
)
clinical$clinical_multiple_treatment = ifelse( clinical$clinical_meta_consolidatedTreatmentType == "multiple", 1,0) 

#### Filter to only response and clinical fields

In [12]:
clinical <- clinical %>% select(sampleId, 
                                patientIdentifier,
                                ID_meta_hmfSampleId, 
                                Filter_meta_responseMeasured, 
                                Y_best_response_binary,
                                Y_best_response, 
                                Y_best_response_time_in_days, 
                                Y_relapse, 
                                Survival_pfs_event, 
                                Survival_time_to_pfs_event,
                                Survival_os_event,
                                Survival_time_to_os_event,
                                Survival_at_6_months,
                                Survival_at_12_months,
                                Survival_at_18_months, 
                                contains("clinical"))

#### 3 - Output the Data 

In [13]:
write.csv( clinical, paste0( TMP_DIR, 'clinical_ready.csv'), row.names = FALSE)